### Chapter 20 속성 디스크립터

+ 디스크립터

디스크립터를 이용하면 여러 속성에 대한 동일한 접근을 재사용할 수 있다. 코드가 좋아진다는 것이다.

디스크립터는 `__get__, __set__, __delete__` 메서드로 구성된 프로토콜을 구현하는 클래스다. property는 이 프로토콜을 완벽히 구현한다. 그러나 프로토콜이 으레 그렇듯 그중 일부만 구현해도 된다. 게터와 세터, 혹은 하나만 구현한 디스크립터도 많다.

+ 디스크립터의 예시

19장에서 프로퍼티 팩토리를 사용해서, 비슷한 형태의 게터와 세터를 추상화하는 방법을 알아보았다. property 함수는 접근자 함수를 매개변수화한 후 환경변수를 클로저에 담아 프로퍼티 객체를 생성한다. 이를 객체지향 방식으로 해결한 게 디스크립터 클래스다.

> `__get__(), __set__(), __delete__()` 메서드를 구현하는 클래스가 디스크립터다. 디스크립터는 클래스의 객체를 다른 클래스의 클래스 속성으로 정의하여 사용한다.



In [2]:
class Quantity:
  def __init__(self, storage_name):
    self.storage_name=storage_name


  def __set__(self, inst, value):
    if value>0:
      inst.__dict__[self.storage_name]=value
      #set 메서드 호출을 통한 무한 재귀를 막기 위해, __dict__를 직접 처리한다.
    else:
      raise ValueError('value must be larger than 0')


class LineItem:
  weight=Quantity('weight')
  price=Quantity('price')
  #디스크립터 객체를 각 속성에 바인딩
  #관리 대상 속성이 저장소 속성과 이름이 같으므로 굳이 게터는 구현할 필요 없다

  def __init__(self, description, weight, price):
    self.description=description
    self.weight=weight
    self.price=price

  
  def subtotal(self):
    return self.price*self.weight

truffle=LineItem('white truffle', 100, 0)
#value가 0이므로 error 발생

그런데 디스크립터 객체 생성시 `weight=Quantity('weight')` 등으로 속성명을 반복해 적어야 하는 것이 불편하다. 게다가 실수로 `weight=Quantity('price')` 따위의 코드를 짠다면 엉뚱한 결과가 발생할 수 있다. 이러한 문제에 대한 임시방편적인 해결책을 설명하고, 21장에서는 제대로 된 해결책을 제시한다. (메타클래스 사용)

In [4]:
class Quantity:
  __counter=0

  def __init__(self):
    cls=self.__class__
    prefix=cls.__name__
    index=cls.__counter
    self.storage_name='_{}#{}'.format(prefix,index)
    #객체가 하나 더 생성될 때마다 storage_name의 해시번호를 1씩 늘려가면서 속성명으로 저장한다
    cls.__counter+=1

  def __get__(self, inst, owner):
    #관리대상 속성의 이름이 storage_name과 다르므로 __get__을 따로 구현해야 한다.
    #owner는 관리 대상 클래스-여기서는 LineItem-에 대한 참조이다
    if inst is None: #객체를 통해 호출 안하면 자기 자신을 반환
      return self
    else:
      return getattr(inst, self.storage_name)

  def __set__(self, inst, value):
    if value>0:
      setattr(inst, self.storage_name, value)
      #inst 안에 속성명과 값을 저장하자
    else:
      raise ValueError('value must be larger than 0')


class LineItem:
  weight=Quantity()
  price=Quantity()

  def __init__(self, description, weight, price):
    self.description=description
    self.weight=weight
    self.price=price

  
  def subtotal(self):
    return self.weight*self.price

coconut=LineItem('coconut', 20, 17.95)
print(coconut.weight, coconut.price)
#예상한 대로 잘 작동한다

20 17.95


그러나 위 코드는 실제 내부적으로는 '_Quantity#0' 따위의 속성명으로 속성이 저장되므로, 사용자가 디버깅하기 어렵다는 단점이 있다. 이런 걸 해결하는 방식은 21장에서 제대로 배운다.

물론 프로퍼티 팩토리 코드를 조금 수정해서(책 예제 20-5 참고) 이런 동작을 가능하게 할 수도 있지만 확장성을 위해서 디스크립터 클래스를 사용하는 게 낫다. 가령 빈 description을 방지하기 위해서는 NonBlank 라는 클래스를 만들어서 검증하는 클래스를 새로 짜는 방법이 있다. (예제 20-6 참고) 이런 확장 부분에서 디스크립터 클래스가 훨씬 나은 것이다.

+ 오버라이딩 디스크립터

`__set__`을 구현하여, 관리대상 객체 안에 있는 동일한 이름의 속성을 오버라이드하도록 하는 디스크립터를 오버라이딩 디스크립터overriding descriptor 라고 한다. 그러나 그렇게 작동하지 않는 논오버라이딩 디스크립터도 존재한다.

+ 파이썬의 속성 처리

보통 객체를 통해 속성을 읽으면 객체에 정의된 속성을 반환하지만 그 속성이 객체에 없으면 클래스 속성을 읽는다. 그리고 객체에 새로운 속성을 만들고 그 값을 할당하면 객체 내부에만 그 속성을 만들고 클래스 속성에는 영향을 미치지 않는다. 이런 비대칭성 때문에 `__set__` 메서드의 유무에 따라 디스크립터의 종류가 갈리는 것이다. 이를 공부하기 위한 코드가 있다.


In [6]:
# 단지 출력만을 위한 함수

def cls_name(obj_or_cls):
  cls=type(obj_or_cls)
  if cls is type:
    cls=obj_or_cls
  return cls.__name__.split('.')[-1]

def display(obj):
  cls=type(obj)
  if cls is type:
    return '<class {}>'.format(obj.__name__)
  elif cls in [type(None), int]:
    return repr(obj)
  else:
    return '<{} object>'.format(cls_name(obj))

def print_args(name, *args):
  pseudo_args=', '.join(display(x) for x in args)
  print('-> {}.__{}__({})'.format(cls_name(args[0]), name, pseudo_args))


class Overriding:
  #오버라이딩 디스크립터
  def __get__(self, inst, owner):
    print_args('get', self, inst, owner)
  
  def __set__(self, inst, value):
    print_args('set', self, inst, value)


class OverridingNoGet:
  #__get__없는 오버라이딩 디스크립터
  def __set__(self, inst, value):
    print_args('set', self, inst, value)


class NonOverriding:
  #논오버라이딩 디스크립터
  def __get__(self, inst, owner):
   print_args('get', self, inst, owner)

class Managed:
  over=Overriding()
  over_no_get=OverridingNoGet()
  non_over=NonOverriding()

  def spam(self):
    print('-> Managed.spam({})'.format(display(self)))

obj=Managed()
print(obj.over)
print(Managed.over)

-> Overriding.__get__(<Overriding object>, <Managed object>, <class Managed>)
None
-> Overriding.__get__(<Overriding object>, None, <class Managed>)
None


다양한 테스트를 책의 770-774쪽에서 진행한다. 결국 객체 속성이 어디를 가리키느냐의 문제인 것이다. `__get__, __set__` 중 어느 걸 구현하냐에 따라, 객체 속성을 읽고 쓸 때 어떤 메서드에 접근하는지가 달라지는 것이다. 

In [8]:
obj=Managed()
print(obj.spam)
print(Managed.spam)
obj.spam=7
print(obj.spam)

<bound method Managed.spam of <__main__.Managed object at 0x7f01ce18efd0>>
<function Managed.spam at 0x7f01ce224950>
7


모든 사용자 정의 함수는 `__get__`메서드가 있어서 디스크립터로 작동한다. 그리고 클래스에 연결된 함수는 디스크립터로 작동하기 때문에 클래스 내부 함수는 클래스에 바인딩된 메서드가 된다. 또 위의 코드를 보면, 함수는 논오버라이딩 디스크립터이다. (`__set__` 미구현)

+ 객체 속성에 의한 오버라이딩과 특별 메서드

위 코드가 가르쳐 주는 것이 하나 더 있는데, 함수와 메서드는 `__get__`만 구현하므로 동일한 이름의 객체 속성에 저장하는 건 간섭하지 않는다는 것이다. 위 코드에서도 `obj.spam=7` 로 설정하자 함수가 바뀐 것을 볼 수 있다. 그러나 이는 특별 메서드에 대해서는 작동하지 않는다. 파이썬 인터프리터는 클래스 자체의 특별 메서드를 먼저 검색하므로, 객체에서 특별 메서드 이름 속성을 오버라이드한다고 해서(가령 a 클래스의 b객체에서 `b.__repr__`을 새로 저장한다든지) 특별 메서드의 동작은 바뀌지 않는다.